# IA based CRO applied to Online Banking Selling Forms
Dataton Galicia 2019

Panuccio Abraham Alan

September of 2019

# Data Preparation

In [2]:
import numpy as np 
import pandas as pd
import copy
from datetime import timedelta
from datetime import datetime
import pandas.core.algorithms as algos
from pandas import Series
import scipy.stats.stats as stats
import string
import functools
from importlib import reload
import functools
from sklearn import preprocessing
import joblib
from google.colab import drive
from importlib.machinery import SourceFileLoader
#import custom_feature_selection as cfs
#import custom_constants as cc

BASE_PATH="/content/gdrive/My Drive/Tesis/"

drive.mount('/content/gdrive/')

cfs = SourceFileLoader("custom_feature_selection", BASE_PATH+"custom_feature_selection.py").load_module()
cc = SourceFileLoader("custom_constants", BASE_PATH+"custom_constants.py").load_module()

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


## First Load & Datasets Union

In [0]:
# Loans Convertions
conversions_2018  = pd.read_csv(BASE_PATH+cc.PATH_CONVERTIONS)
conversions_2019 = pd.read_csv(BASE_PATH+cc.PATH_CONVERTIONS_2019)

# Start unifying both Convertion datasets
conversions = pd.concat([conversions_2018, conversions_2019], sort=False)
del conversions_2018
del conversions_2019

# PageViews
data_0 = pd.read_csv(BASE_PATH + cc.PATH_PAGE_VIEWS, parse_dates=["FEC_EVENT"])
data_1 = pd.read_csv(BASE_PATH + cc.PATH_PAGE_VIEWS2, parse_dates=["FEC_EVENT"])

# PageCategories (CONTENT, BOTTOM & TOP)
content_category = pd.read_csv(BASE_PATH + cc.PATH_CONTENT_CATEGORY)
content_category_top = pd.read_csv(BASE_PATH + cc.PATH_CONTENT_CATEGORY_TOP)
content_category_bottom = pd.read_csv(BASE_PATH + cc.PATH_CONTENT_CATEGORY_BOTTOM)

# Clients Connection Settings
device_data = pd.read_csv(BASE_PATH + cc.PATH_DEVICE_DATA, parse_dates=["FEC_EVENT"])
device_data[['USER_ID']]=device_data['USER_ID'].astype(int)

# Sites & Pages Details
site_id = pd.read_csv(BASE_PATH + cc.PATH_SITE)
page = pd.read_csv(BASE_PATH + cc.PATH_PAGE)

In [4]:
# Start unifying both pageviews datasets
data = pd.concat([data_0, data_1], sort=False)
del data_0
del data_1

# For data sample
#data_clientes=pd.DataFrame(data['USER_ID'].drop_duplicates()).head(1000)
#data = data_clientes.merge(data, how="left", on=["USER_ID"])

print(data.shape)

data.head(5)

(22870354, 8)


,FEC_EVENT,PAGE,CONTENT_CATEGORY,CONTENT_CATEGORY_TOP,CONTENT_CATEGORY_BOTTOM,SITE_ID,ON_SITE_SEARCH_TERM,USER_ID
0,2018-03-30 07:35:48,1,1,1,1,1,1,0
1,2018-03-30 07:35:52,2,2,2,2,2,1,0
2,2018-03-30 07:36:11,3,2,2,2,3,1,0
3,2018-03-30 07:36:16,4,2,2,2,3,1,0
4,2018-03-30 07:41:38,5,2,2,2,2,1,0


## Date & Time Features Formating
We will create features to separate the `MONTH` and `WEEKEND`.

In [5]:
# Date Formating & Time Spliting (DAY_OF_MONTH, MONTH, WEEKEND)
data['JUST_DATE']=data['FEC_EVENT'].dt.strftime("%Y%m%d").astype(int)

# Cast the JUST_DATE column to timestamp to do the join
data["JUST_DATE"] = list(map(lambda datee: datetime(int(str(datee)[:4]), int(str(datee)[4:6]), int(str(datee)[6:8])), data["JUST_DATE"]))

data['MONTH'] = data['FEC_EVENT'].dt.month
data['WEEKEND'] =np.where( data['FEC_EVENT'].dt.weekday>=5,'WEEKEND','WEEK')
data['ALL']=1

data.head(5)

,FEC_EVENT,PAGE,CONTENT_CATEGORY,CONTENT_CATEGORY_TOP,CONTENT_CATEGORY_BOTTOM,SITE_ID,ON_SITE_SEARCH_TERM,USER_ID,JUST_DATE,MONTH,WEEKEND,ALL
0,2018-03-30 07:35:48,1,1,1,1,1,1,0,2018-03-30,3,WEEK,1
1,2018-03-30 07:35:52,2,2,2,2,2,1,0,2018-03-30,3,WEEK,1
2,2018-03-30 07:36:11,3,2,2,2,3,1,0,2018-03-30,3,WEEK,1
3,2018-03-30 07:36:16,4,2,2,2,3,1,0,2018-03-30,3,WEEK,1
4,2018-03-30 07:41:38,5,2,2,2,2,1,0,2018-03-30,3,WEEK,1


## Users vs Months Cross Table
Build a `Cross Table` with al `USER_ID` in all `MONTH`, to complete the users whom didn't visited the site in a month.

In [6]:
# Customers & Months' keys in differents datasets
dt_clients = pd.DataFrame(data['USER_ID'].drop_duplicates())
dt_months  = pd.DataFrame(data['MONTH'].drop_duplicates())

# Dummy column to do the Cartersian Product
dt_clients['DUMMY'] = 0
dt_months['DUMMY'] = 0

# Outer Join (ex data_completo)
dt_universe = dt_clients.merge(dt_months, how='outer', on=["DUMMY"])
dt_universe.set_index(['USER_ID','MONTH'])

# Drop dummy column
dt_universe.drop("DUMMY", axis=1, inplace=True)

# Final counts
dt_universe.groupby(["MONTH"])[["USER_ID"]].count()

,USER_ID
MONTH,
1,11676
2,11676
3,11676
4,11676
5,11676
6,11676
7,11676
8,11676
9,11676


## Conversions Dataset - Target 3 Months
We build a unified dataset with al `USER_ID` & `MONTHS` combinations and the conversion flag in the next three months `TARGET` which is the target of prediction.

What's more, we created four more features with the past convertions to catch the Loans' re-takers. Those features see the three months before & the current month, the features name are: 

`CONVERSION_CURR`: About the current month

`CONVERSION_L1`: About the month -1

`CONVERSION_L2`: About the month -2

`CONVERSION_L3`: About the month -3

`CONVERSION_LU`: Unify the months -1, -2 or -3 in just one feature

In [25]:
# A fix to add the 2019 conversions as the 13 month
conversions.groupby(["mes","anio"]).count()
conversions["mes"][conversions.anio==2019] = [13.0 for i in range(len(conversions[conversions.anio==2019].mes))]
conversions.groupby(["mes","anio"]).count()

,,USER_ID
mes,anio,
1.0,2018.0,89
2.0,2018.0,139
3.0,2018.0,49
4.0,2018.0,165
5.0,2018.0,105
6.0,2018.0,196
7.0,2018.0,102
8.0,2018.0,213
9.0,2018.0,171


In [26]:
# Bakcup from the original dataset
conversions2 = copy.copy(conversions)
# Convert USER_ID to integer
conversions2['USER_ID'] = conversions2.USER_ID.astype(int)
# Just need a row per client
conversions2 = conversions2.drop_duplicates()
# We start with all USER_ID & MONTH combinations
dt_conv_unify = copy.copy(dt_universe[['USER_ID','MONTH']])
    
# New features configuration, iterate for each conversion time's window to build each features
new_features = {"P1":-1, "P2":-2, "P3":-3,"L1":1, "L2":2, "L3":3, "CURR":0}
for l in new_features.keys():
    v = new_features[l]
    
    # Add columns for each period
    conversions2["MONTH_"+l]   = conversions2.mes.astype(int)+v
    
    # The target variable always in 1
    conversions2['CONVERT']=1

    # Now build a different dataset for each new feature to add
    dt_conv_temp = pd.crosstab([conversions2["USER_ID"],conversions2["MONTH_"+l]], conversions2.CONVERT)
    dt_conv_temp.index.names=['USER_ID','MONTH']
    dt_conv_temp.columns=['CONVERSION_'+l]

    # Unify the datasets in just one with the differents kind of conversions (+1, +2, +3, Current, -1, -2, -3)
    dt_conv_unify = dt_conv_unify.merge (dt_conv_temp, how='left', on=['USER_ID','MONTH'])
    
    # Delete what I don't need
    del dt_conv_temp
    
    # Set as 0 the clients which hasn't converted in each month
    dt_conv_unify['CONVERSION_'+l].fillna(0,inplace=True)
    
# Delete what I don't need
del conversions2

# Target feature with the -1 or -2 o -3 conversions unified
dt_conv_unify['CONVERSION_LU'] = (dt_conv_unify['CONVERSION_L1'] + dt_conv_unify['CONVERSION_L2'] + dt_conv_unify['CONVERSION_L3'])

# Target feature with the +1 or +2 o +3 conversions unified
dt_conv_unify['TARGET'] = np.where((dt_conv_unify['CONVERSION_P1'] + dt_conv_unify['CONVERSION_P2'] + dt_conv_unify['CONVERSION_P3']) > 0, 1, 0)

# Drop false predictor columns
dt_conv_unify.drop(['CONVERSION_P1','CONVERSION_P2','CONVERSION_P3'], axis=1, inplace=True)

# Write the dataset with conversions features
dt_conv_unify.to_csv(BASE_PATH + cc.PATH_DT_CONVERSIONS)

dt_conv_unify.groupby(["MONTH","TARGET"])[["USER_ID"]].count()

USER_ID
MONTH TARGET         
1     0         11367
      1           309
2     0         11391
      1           285
3     0         11302
      1           374
4     0         11351
      1           325
5     0         11285
      1           391
6     0         11284
      1           392
7     0         11219
      1           457
8     0         11252
      1           424
9     0         11285
      1           391
10    0         11217
      1           459
11    0         11329
      1           347
12    0         11414
      1           262

In [27]:
dt_conv_unify.head(10)

,USER_ID,MONTH,CONVERSION_L1,CONVERSION_L2,CONVERSION_L3,CONVERSION_CURR,CONVERSION_LU,TARGET
0,0,3,0.0,0.0,0.0,0.0,0.0,0
1,0,2,0.0,0.0,0.0,0.0,0.0,0
2,0,1,0.0,0.0,0.0,0.0,0.0,0
3,0,5,0.0,0.0,0.0,0.0,0.0,0
4,0,6,0.0,0.0,0.0,0.0,0.0,0
5,0,4,0.0,0.0,0.0,0.0,0.0,0
6,0,11,0.0,0.0,0.0,0.0,0.0,0
7,0,12,0.0,0.0,0.0,0.0,0.0,0
8,0,9,0.0,0.0,0.0,0.0,0.0,0
9,0,10,0.0,0.0,0.0,0.0,0.0,0


## Custom Pages Categories and Hierarchy Level
To get a better understanding about the customer navegation patterns, we create two custom categories for each page.


The first category `PAGE_OWN` is a custom business category, built focused in the convertion target (personal loans), to make a better distinction between a loan page, a loan request page and others pages...


The second category `PAGE_HIERARCHY` is a hierarchical category, based on the site structure & the page level on the site map. What's more it makes a distinction for APP and WEB site paths.


In [0]:
page = pd.read_csv(BASE_PATH + cc.PATH_PAGE)
# User-defined function to split the hierarchical level of the page's path with a defined delimiter
def parse_final_hierarchy(page_path, delimiters):
    delimiter = ':'
    
    # If the text starts with a delimiter, it will be deleted
    if page_path.startswith('/'):
        page_path = page_path.replace('/', '', 1)
    
    # Replace all differents delimiters by the standar delimiter
    for d2 in delimiters.keys():
        d = delimiters[d2]
        page_path = page_path.replace(d, delimiter)

    # Split the page_path by the default delimiter
    try:
        splits = page_path.split(delimiter)
        r = splits[len(splits)-1]
        if r =='':
            return 'OTHERS'
        else: 
            return r
    except :
        return 'OTHERS'

# User-defined function to define the own custom business category for each page
def build_categories(dtm2):
    delimiters = {'APP':':', 'WEB':'/'}

    # For each column in the dataset
    for i in range(len(dtm2.dtypes)):
        col = str(dtm2.columns[i])

        # Just take the strings columns wich have a "descripcion" literal in the text content
        if 'descripcion' in col and ('str' in str(dtm2.dtypes[i]) or 'object'  in str(dtm2.dtypes[i])):            
            
            # Own Category
            dtm2[str(col.replace('_descripcion', '')+'_OWN').upper()] = list(map(lambda texto: 'REQUEST_LOAN' if ( ('SOLI' in texto.upper() or 'ALTA' in texto.upper()) and ('PREST' in texto.upper() and not('AUTO' in texto.upper()) and not('PYME' in texto.upper()) and not('HIPO' in texto.upper()) ) ) else 'PLOANS' if ('PREST' in texto.upper() and not('AUTO' in texto.upper()) and not('PYME' in texto.upper()) and not('HIPO' in texto.upper()) ) else 'LOANS' if ('PREST' in texto.upper()) else 'EMINENT' if ('EMINENT' in texto.upper()) else 'TRANSFERS' if ('TRANSF' in texto.upper()) else 'INVESTMENTS' if ('INVER' in texto.upper()) else 'INSURANCE' if ('SEGU' in texto.upper()) else 'LOG' if ('LOG' in texto.upper()) else 'OTHERS', dtm2[col]))
            
            # Count the delimiters' quantity present in the page_path
            jerarquia = 0
            for d2 in delimiters.keys():
                d = delimiters[d2]
                jerarquia += max(list(map(lambda texto: len(texto.split(d)) if (d in texto.upper()) else 0, dtm2[col])))
            
            # Check that the page_path have, at least, one delimiter to split
            if jerarquia > 0:
                # Define the hierarchical level category
                dtm2[str(col.replace(' ','_').replace('_descripcion', '')+'_HIERARCHY').upper()] = list(map(lambda texto: parse_final_hierarchy(texto, delimiters), dtm2[col]))

    return dtm2

# Create user-defined categories
page = build_categories(page)

# Delete the original "PAGE_descripcion" column
page.drop(['PAGE_descripcion'], axis=1, inplace=True)

page.groupby(["PAGE_OWN"])[["PAGE_OWN"]].count()

,PAGE_OWN
PAGE_OWN,
EMINENT,124
INSURANCE,80
INVESTMENTS,94
LOANS,19
LOG,70
OTHERS,1137
PLOANS,37
REQUEST_LOAN,11
TRANSFERS,263


In [0]:
dt = page.groupby(["PAGE_HIERARCHY"])[["PAGE"]].count()
dt.columns=["PAGES"]
dt.sort_values(by=["PAGES"], ascending=False).head(15)

,PAGES
PAGE_HIERARCHY,
PRODUCTOSYSERVICIOS,387
INFORMACION-CONTACTO,180
BENEFICIOSYPROMOCIONES,144
TARJETAS,85
ERRORES,84
INVERSIONES,71
PERFIL,64
TRANSFERENCIAS,48
SEGURIDAD,41


## Holidays Dates Features
Create features which catch the temporal context of the page's view, in this case we built one new feature wich embebed the behavior of the cliente the days before, during & after a Holiday. This idea is based on the common use of the loans to support the Holidays charges, it can be before, after or during the Holidays.

To do it we will add a new dataset `holidays_2018.csv` which include all Holidays in 2018.

The new features created are:

`HOLIDAY_UNIFIED`: Where we let a unique flag in case of the visit day was before, after or during a holiday.
    
`HOLIDAY_OPEN`: Where we let a different flag for each case: before, after, during a holiday and others.

In [0]:
# Add a Holidays' dataset for all the year (2018)
dt_holidays = pd.read_csv(BASE_PATH + cc.PATH_HOLIDAYS_2018, sep=";", parse_dates=["JUST_DATE"])
feriados_orig = copy.copy(dt_holidays)
dt_holidays.head(5)

,JUST_DATE,DESCRIPCION
0,2018-01-01,Anio Nuevo
1,2018-12-02,Carnaval
2,2018-02-13,Carnaval
3,2018-03-29,Jueves Santo Festividad Cristiana
4,2018-03-30,Viernes Santo Festividad Cristiana


### Holiday Date Feature

In [0]:
# Flag for the holiday dates
data=data.merge(dt_holidays, how='left')
data["HOLIDAY"] = list(map(lambda is_holiday: "HOLIDAY" if is_holiday == is_holiday else "NOT_HOLIDAY", data["DESCRIPCION"]))
data.drop(['DESCRIPCION'], axis=1, inplace=True)
data.groupby(["HOLIDAY"])[["USER_ID"]].count()

,USER_ID
HOLIDAY,
HOLIDAY,723075
NOT_HOLIDAY,22147279


### Before Holidays Dates Features

In [0]:
# Flag for the N dates before a holiday
CN_DAYS_BEFOREHOLIDAY = 4
data["BEFORE_HOLIDAY"] =""
for i in range(CN_DAYS_BEFOREHOLIDAY-1):
    dt_holidays['JUST_DATE']= (dt_holidays.JUST_DATE + timedelta(days=-1))
    data=data.merge(dt_holidays, how='left')
    data["BEFORE_HOLIDAY"] = list(map(lambda is_holiday, before_holiday: "BEFORE_HOLIDAY" if (is_holiday == is_holiday or before_holiday =="BEFORE_HOLIDAY") else "NOT_HOLIDAY", data["DESCRIPCION"], data["BEFORE_HOLIDAY"]))
    data.drop(['DESCRIPCION'], axis=1, inplace=True)

data.groupby(["BEFORE_HOLIDAY"])[["USER_ID"]].count()

,USER_ID
BEFORE_HOLIDAY,
BEFORE_HOLIDAY,2697459
NOT_HOLIDAY,20172895


### After Holidays Dates Features

In [0]:
data["AFTER_HOLIDAY"] =""

# Return to the original dates
dt_holidays['JUST_DATE']= (dt_holidays.JUST_DATE + timedelta(days=CN_DAYS_BEFOREHOLIDAY+1))

# Flag for the N dates after a holiday
data["AFTER_HOLIDAY"] =""
for i in range(CN_DAYS_BEFOREHOLIDAY-1):
    dt_holidays['JUST_DATE']= (dt_holidays.JUST_DATE + timedelta(days=1))
    data=data.merge(dt_holidays, how='left')
    data["AFTER_HOLIDAY"] = list(map(lambda is_holiday, after_holiday: "AFTER_HOLIDAY" if (is_holiday == is_holiday or after_holiday =="AFTER_HOLIDAY") else "NOT_HOLIDAY", data["DESCRIPCION"], data["AFTER_HOLIDAY"]))
    data.drop(['DESCRIPCION'], axis=1, inplace=True)

data.groupby(["AFTER_HOLIDAY"])[["USER_ID"]].count()

,USER_ID
AFTER_HOLIDAY,
AFTER_HOLIDAY,3422785
NOT_HOLIDAY,19447569


### Unifying Holidays Dates Features - During, Before & After 

In [0]:
# Unifying the three features in two: HOLIDAY_UNIFIED and HOLIDAY_OPEN features
data["HOLIDAY_UNIFIED"] = list(map(lambda is_holiday, after_holiday, before_holiday: "HOLIDAY" if (is_holiday=="HOLIDAY") else "HOLIDAY" if (after_holiday=="AFTER_HOLIDAY") else "HOLIDAY" if (before_holiday=="BEFORE_HOLIDAY") else "NOT_HOLIDAY" , data["HOLIDAY"], data["AFTER_HOLIDAY"], data["BEFORE_HOLIDAY"]))
data["HOLIDAY_OPEN"] = list(map(lambda is_holiday, after_holiday, before_holiday: "HOLIDAY" if (is_holiday=="HOLIDAY") else "AFTER_HOLIDAY" if (after_holiday=="AFTER_HOLIDAY") else "BEFORE_HOLIDAY" if (before_holiday=="BEFORE_HOLIDAY") else "NOT_HOLIDAY" , data["HOLIDAY"], data["AFTER_HOLIDAY"], data["BEFORE_HOLIDAY"]))
data.drop(['BEFORE_HOLIDAY'], axis=1, inplace=True)
data.drop(['AFTER_HOLIDAY'], axis=1, inplace=True)
data.drop(['HOLIDAY'], axis=1, inplace=True)

data.groupby(["HOLIDAY_UNIFIED"])[["USER_ID"]].count()

,USER_ID
HOLIDAY_UNIFIED,
HOLIDAY,6408908
NOT_HOLIDAY,16461446


In [0]:
data.groupby(["HOLIDAY_OPEN"])[["USER_ID"]].count()

,USER_ID
HOLIDAY_OPEN,
AFTER_HOLIDAY,3328358
BEFORE_HOLIDAY,2357475
HOLIDAY,723075
NOT_HOLIDAY,16461446


In [0]:
# Merge with the original dataset
del feriados_orig
del dt_holidays
data=data.merge(page, how='left', on=['PAGE'])
data.head(5)

,FEC_EVENT,PAGE,CONTENT_CATEGORY,CONTENT_CATEGORY_TOP,CONTENT_CATEGORY_BOTTOM,SITE_ID,ON_SITE_SEARCH_TERM,USER_ID,JUST_DATE,MONTH,WEEKEND,ALL,HOLIDAY_UNIFIED,HOLIDAY_OPEN,PAGE_OWN,PAGE_HIERARCHY
0,2018-03-30 07:35:48,1,1,1,1,1,1,0,2018-03-30,3,WEEK,1,HOLIDAY,HOLIDAY,EMINENT,WEB
1,2018-03-30 07:35:52,2,2,2,2,2,1,0,2018-03-30,3,WEEK,1,HOLIDAY,HOLIDAY,LOG,LOGIN
2,2018-03-30 07:36:11,3,2,2,2,3,1,0,2018-03-30,3,WEEK,1,HOLIDAY,HOLIDAY,OTHERS,INICIO
3,2018-03-30 07:36:16,4,2,2,2,3,1,0,2018-03-30,3,WEEK,1,HOLIDAY,HOLIDAY,OTHERS,CUENTAS
4,2018-03-30 07:41:38,5,2,2,2,2,1,0,2018-03-30,3,WEEK,1,HOLIDAY,HOLIDAY,OTHERS,ERROR


In [0]:
# Unifying a LOANS/HOLIDAYS feature to catch the Loans requested by Holidays costs.
data["HOLIDAY_LOANS"] = list(map(lambda is_holiday, category: "HOLIDAY_"+category if (is_holiday !="NOT_HOLIDAY" and 'LOAN' in category) else "NOT_HOLIDAY" , data["HOLIDAY_UNIFIED"], data["PAGE_OWN"]))
data.groupby(["HOLIDAY_LOANS"])[["USER_ID"]].count()

,USER_ID
HOLIDAY_LOANS,
HOLIDAY_LOANS,5938
HOLIDAY_PLOANS,66644
HOLIDAY_REQUEST_LOAN,82098
NOT_HOLIDAY,22715674


In [0]:
# Clean the RAM (XD)
import gc

del page
del content_category
del content_category_top
del content_category_bottom
del device_data
del site_id

gc.collect()

2205

## Historical Features

Now we will build historical features which consider time's windows of 4, 5 & 6 months from the original dataset. So, we will build two kinds of features for each column in the original dataset (ignoring the `columns_to_ignore`):

`4M_XXCOL_XXVAL`: % of visits what the user did in pages categorized as XXVAL in the column XXCOL in the last 4 months.

`4M_XXCOL_UQ_XXVAL`: Quantity of unique days when the user visited the pages categorized as XXVAL in the column XXCOL in the last 4 months.

`5M_XXCOL_XXVAL`: % of visits what the user did in pages categorized as XXVAL in the column XXCOL in the last 5 months.

`5M_XXCOL_UQ_XXVAL`: Quantity of unique days when the user visited the pages categorized as XXVAL in the column XXCOL in the last 5 months.

`6M_XXCOL_XXVAL`: % of visits what the user did in pages categorized as XXVAL in the column XXCOL in the last 6 months.

`6M_XXCOL_UQ_XXVAL`: Quantity of unique days when the user visited the pages categorized as XXVAL in the column XXCOL in the last 6 months.

In [0]:
# User-defined function which build the historical features for the dataset, between the start and end months
# for each feature in the dataset (ignoring columns_to_ignore) calculate the % of visits in each category
# of the feature and the quantity of unique days when the user visited this category.
def historical_features(dataset, start_month, end_month, columns_to_ignore, time_window, q_months):
    def conjunction(*conditions):
        return functools.reduce(np.logical_and, conditions)
    
    q_days = (((end_month - start_month)+1) * 30)    
    
    # Filtering the requested months
    c1 = dataset['FEC_EVENT'].dt.month >= start_month
    c2 = dataset['FEC_EVENT'].dt.month <= end_month
    data_temp = dataset[conjunction(c1,c2)]
    
    # Initializing the result dataset
    dt_result = []
    q_columns = len(data_temp.drop(columns_to_ignore, axis=1).columns)
    q_col = 0
    global q_max2
    # For each column we will build the historical features
    for c in data_temp.drop(columns_to_ignore, axis=1).columns:
        q_col = (q_col+1)
        if int(((q_col*end_month)/(q_columns*q_months))*10) > q_max2:
            q_max2 = int(((q_col*end_month)/(q_columns*q_months))*10)
            print(str(int(q_max2*10))+"% ", end = '')

        # For each USER_ID we calculate the % of visits per category of the column
        temp         = pd.crosstab([data_temp.USER_ID], data_temp[c])
        columnas     = temp.columns    
        temp.columns = [str(time_window)+ "M_" + c + "_" + str(v) for v in temp.columns]
        dt_result.append(temp.apply(lambda x: (round(100000 * (x / x.sum()), 0)), axis=1))

        # For each USER_ID we calculate the quantity of unique days when the user visited each category of the column
        data_temp_unique    = data_temp[['USER_ID','JUST_DATE',c]].drop_duplicates()
        temp_unique         = pd.crosstab([data_temp_unique.USER_ID], data_temp_unique[c]).astype(int)    
        columnas_uniq       = temp_unique.columns    
        temp_unique.columns = [str(time_window)+ "M_" + c + "_UQ_" + str(w) for w in columnas_uniq]
        dt_result.append(temp_unique.apply(lambda x: x, axis=1))
    
    del data_temp
    dt_result = pd.concat(dt_result, axis=1)
    
    return dt_result

In [0]:
# User-defined function to build the historical features for a time's window
def historical_features_by_time_window(tw, path_tw, nbest_feature, acum):
    global dt_tw_unified
    global cols_to_ignore
    global dt_to_use
    global results 
    global q_max2
    global flag_tw
    
    dt_results = []

    # We will process one month at time.
    print("[START] - Historical Features for "+str(tw)+" Months ")
    print("     Feature Build     [ ", end = '')
    q_max2 = 0
    flag = 1
    for m in dt_to_use["MONTH"].unique():
        # Build the USER_ID list for this month
        dt_month = dt_to_use[dt_to_use["MONTH"] == m]

        # Build the Historical features
        dt_6months = historical_features(data, (m-(tw-1)), m, cols_to_ignore, tw, len(dt_to_use["MONTH"].unique()))

        # Join both
        dt_month = dt_month.merge(dt_6months, how='left', on=["USER_ID"])
        dt_month = dt_month.fillna(0)

        # Concat the month to the complete dataset
        if flag==1:
            dt_results = dt_month
            flag = 0
        else:
            dt_results = pd.concat([dt_results, dt_month])    
    print (" ] ")
    
    # Select the training months to use in feature selection
    dt_result_temp = dt_results[((dt_results.MONTH.isin(cc.K_FESELECTION_MONTHS)))]

    # Scoring features focused in the conversion target
    print("     Feature Selection [ ", end = '')
    final_iv, scored_fe = cfs.score_features(dt_result_temp, dt_result_temp.TARGET)
    scored_fe   = scored_fe.rename(columns={'VAR_NAME':'index'})
    selected_fe = scored_fe.sort_values(['IV'],ascending=0).head(nbest_feature)
    selected_fe = selected_fe.set_index('index').index
    
    # Always we need the index columns to do the join after
    col_index = pd.Index(["USER_ID","MONTH"]) 
    col_index = col_index.append(selected_fe)
    col_index = list(set(col_index))
    
    # At the end we keep just the selected columns and the index columns
    dt_results = dt_results[col_index]
    
    # Keep a copy of the features selected
    selected_features[tw] = col_index
    print(" ] ")
    
    # Drop duplicated columns
    dt_results = dt_results.loc[:,~dt_results.columns.duplicated()]
    
    # Write the dataset with the historical features
    dt_results.to_csv(BASE_PATH + path_tw)
    
    # Concat the time's window dataset to the complete dataset
    if acum:
      if flag_tw==1:
          dt_tw_unified = dt_to_use.merge(dt_results, how='left', on=["USER_ID","MONTH"])
          flag_tw = 0
      else:
          dt_tw_unified = dt_tw_unified.merge(dt_results, how='left', on=["USER_ID","MONTH"])

    # Delete what I don't need
    del dt_results

    print ("[ENDED] - Historical Features for "+str(tw)+" Months\n")

In [0]:
# We will process one Time's window at a time, the Time's windows defined are 4, 5 & 6

# The final dataset which unify all time's windows features
global dt_tw_unified
global cols_to_ignore
global dt_to_use
global results 
global q_max2
global flag_tw

dt_tw_unified = []
results = []
selected_features = dict()
flag_tw = 1
q_max2 = 0

# Define the columns we wont use to build historical features, like the MONTH, PAGE, etc...
cols_to_ignore = ["USER_ID", "FEC_EVENT","JUST_DATE","MONTH", "PAGE", "ALL"]

# Start from the completed Cross Table of USER_ID & MONTH
dt_to_use = dt_conv_unify[["USER_ID","MONTH","TARGET"]].sort_values(['MONTH','USER_ID'],ascending=True)

In [0]:
# 4 months time's window
historical_features_by_time_window(4, cc.PATH_DT_4MONTHS, int(round(cc.K_BEST_FEATURES/4)), True)

dt_tw_unified.head(5)

[START] - Historical Features for 4 Months 
     Feature Build     [ 10% 20% 30% 40% 50% 60% 70% 80% 90% 100%  ] 
     Feature Selection [ 10% 20% 30% 40% 50% 60% 70% 80% 90% 100%  ] 
[ENDED] - Historical Features for 4 Months



,USER_ID,MONTH,TARGET,4M_PAGE_HIERARCHY_TARJETAS,4M_PAGE_HIERARCHY_INSTITUCIONAL,4M_ON_SITE_SEARCH_TERM_UQ_40,4M_HOLIDAY_LOANS_UQ_HOLIDAY_PLOANS,4M_CONTENT_CATEGORY_60,4M_PAGE_HIERARCHY_UQ_PERFIL,4M_PAGE_OWN_UQ_PLOANS,4M_CONTENT_CATEGORY_BOTTOM_18,4M_PAGE_HIERARCHY_UQ_TARJETAS,4M_PAGE_HIERARCHY_CONSUMOS,4M_PAGE_HIERARCHY_UQ_PERSONAS,4M_HOLIDAY_OPEN_HOLIDAY,4M_PAGE_HIERARCHY_USERS,4M_PAGE_HIERARCHY_INFORMACION-CONTACTO,4M_PAGE_HIERARCHY_ PASO 1,4M_PAGE_HIERARCHY_UQ_CONSUMOS,4M_CONTENT_CATEGORY_BOTTOM_UQ_9,4M_HOLIDAY_UNIFIED_UQ_HOLIDAY,4M_CONTENT_CATEGORY_BOTTOM_UQ_6,4M_PAGE_HIERARCHY_CUENTAS,4M_PAGE_HIERARCHY_RRHH,4M_CONTENT_CATEGORY_BOTTOM_UQ_60,4M_PAGE_OWN_LOANS,4M_ON_SITE_SEARCH_TERM_UQ_228,4M_HOLIDAY_LOANS_UQ_NOT_HOLIDAY,4M_CONTENT_CATEGORY_6,4M_CONTENT_CATEGORY_3,4M_PAGE_HIERARCHY_ERROR,4M_ON_SITE_SEARCH_TERM_UQ_215,4M_ON_SITE_SEARCH_TERM_UQ_2,4M_ON_SITE_SEARCH_TERM_UQ_293,4M_CONTENT_CATEGORY_34,4M_ON_SITE_SEARCH_TERM_UQ_34,4M_PAGE_HIERARCHY_CONSUMOS-TARJETA-DEBITO,4M_CONTENT_CATEGORY_BOTTOM_13,4M_ON_SITE_SEARCH_TERM_44,4M_PAGE_HIERARCHY_UQ_MODIFICAR-CUENTA-PRIMARIA,...,4M_ON_SITE_SEARCH_TERM_UQ_231,4M_ON_SITE_SEARCH_TERM_UQ_43,4M_HOLIDAY_LOANS_UQ_HOLIDAY_REQUEST_LOAN,4M_ON_SITE_SEARCH_TERM_179,4M_CONTENT_CATEGORY_BOTTOM_UQ_16,4M_PAGE_HIERARCHY_MENSAJE,4M_ON_SITE_SEARCH_TERM_UQ_211,4M_CONTENT_CATEGORY_TOP_7,4M_PAGE_HIERARCHY_TERMINOSYCONDICIONES,4M_HOLIDAY_OPEN_UQ_HOLIDAY,4M_PAGE_HIERARCHY_UQ_CUENTAS,4M_PAGE_HIERARCHY_UQ_RRHH,4M_PAGE_HIERARCHY_RESUMEN,4M_CONTENT_CATEGORY_TOP_UQ_2,4M_HOLIDAY_UNIFIED_UQ_NOT_HOLIDAY,4M_CONTENT_CATEGORY_UQ_60,4M_ON_SITE_SEARCH_TERM_UQ_227,4M_CONTENT_CATEGORY_27,4M_ON_SITE_SEARCH_TERM_1,4M_CONTENT_CATEGORY_UQ_16,4M_PAGE_HIERARCHY_UQ_INICIO,4M_PAGE_HIERARCHY_PRESTAMOS,4M_HOLIDAY_OPEN_UQ_AFTER_HOLIDAY,4M_PAGE_OWN_UQ_LOANS,4M_PAGE_HIERARCHY_UQ_AYUDA,4M_CONTENT_CATEGORY_BOTTOM_7,4M_CONTENT_CATEGORY_BOTTOM_6,4M_PAGE_HIERARCHY_LANDING,4M_ON_SITE_SEARCH_TERM_UQ_219,4M_ON_SITE_SEARCH_TERM_UQ_33,4M_PAGE_OWN_UQ_OTHERS,4M_CONTENT_CATEGORY_BOTTOM_3,4M_HOLIDAY_LOANS_UQ_HOLIDAY_LOANS,4M_ON_SITE_SEARCH_TERM_120,4M_CONTENT_CATEGORY_TOP_3,4M_PAGE_HIERARCHY_UQ_ PASO 1,4M_CONTENT_CATEGORY_2,4M_CONTENT_CATEGORY_BOTTOM_60,4M_CONTENT_CATEGORY_11,4M_CONTENT_CATEGORY_BOTTOM_23
0,0,1,0,282.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,1.0,141.0,0.0,7746.0,0.0,0.0,282.0,1.0,0.0,9.0,1.0,845.0,NaN,NaN,0.0,0.0,26.0,141.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,2.0,1.0,NaN,0.0,25.0,17.0,NaN,0.0,0.0,100000.0,0.0,2.0,0.0,5.0,0.0,0.0,0.0,141.0,NaN,0.0,0.0,2.0,0.0,0.0,0.0,0.0,1.0,4225.0,NaN,0.0,0.0
1,1,1,0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,10000.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,NaN,NaN,0.0,0.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,1.0,0.0,NaN,0.0,13.0,10.0,NaN,0.0,0.0,100000.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,10400.0,NaN,0.0,0.0
2,2,1,0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,2362.0,0.0,3937.0,0.0,0.0,0.0,1.0,0.0,2.0,0.0,1575.0,NaN,NaN,0.0,0.0,11.0,0.0,0.0,2362.0,0.0,0.0,0.0,NaN,0.0,0.0,2362.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,1575.0,1.0,2.0,NaN,2362.0,11.0,9.0,NaN,0.0,0.0,100000.0,0.0,5.0,0.0,1.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,35433.0,NaN,1575.0,0.0
3,3,1,0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,1.0,21368.0,0.0,0.0,0.0,0.0,1.0,6.0,0.0,0.0,NaN,NaN,0.0,0.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,2.0,0.0,NaN,0.0,15.0,9.0,NaN,0.0,0.0,100000.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,14.0,0.0,0.0,0.0,0.0,0.0,4274.0,NaN,0.0,0.0
4,4,1,0,7717.0,0.0,0.0,0.0,NaN,0.0,1.0,965.0,3.0,5788.0,0.0,965.0,0.0,0.0,965.0,4.0,0.0,7.0,4.0,0.0,NaN,NaN,0.0,0.0,17.0,1929.0,0.0,2894.0,0.0,0.0,0.0,NaN,0.0,0.0,322.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,1.0,0.0,NaN,965.0,17.0,10.0,NaN,0.0,0.0,100000.0,0.0,5.0,0.0,4.0,0.0,0.0,322.0,1929.0,NaN,0.0,0.0,8.0,0.0,0.0,0.0,322.0,2.0,43730.0,NaN,0.0,0.0


In [0]:
# 6 months time's window
historical_features_by_time_window(6, cc.PATH_DT_6MONTHS, cc.K_BEST_FEATURES, True)

dt_tw_unified.head(5)

[START] - Historical Features for 6 Months 
     Feature Build     [ 10% 20% 30% 40% 50% 60% 70% 80% 90% 100%  ] 
     Feature Selection [ 10% 20% 30% 40% 50% 60% 70% 80% 90% 100%  ] 
[ENDED] - Historical Features for 6 Months



,USER_ID,MONTH,TARGET,4M_PAGE_HIERARCHY_TARJETAS,4M_PAGE_HIERARCHY_INSTITUCIONAL,4M_ON_SITE_SEARCH_TERM_UQ_40,4M_HOLIDAY_LOANS_UQ_HOLIDAY_PLOANS,4M_CONTENT_CATEGORY_60,4M_PAGE_HIERARCHY_UQ_PERFIL,4M_PAGE_OWN_UQ_PLOANS,4M_CONTENT_CATEGORY_BOTTOM_18,4M_PAGE_HIERARCHY_UQ_TARJETAS,4M_PAGE_HIERARCHY_CONSUMOS,4M_PAGE_HIERARCHY_UQ_PERSONAS,4M_HOLIDAY_OPEN_HOLIDAY,4M_PAGE_HIERARCHY_USERS,4M_PAGE_HIERARCHY_INFORMACION-CONTACTO,4M_PAGE_HIERARCHY_ PASO 1,4M_PAGE_HIERARCHY_UQ_CONSUMOS,4M_CONTENT_CATEGORY_BOTTOM_UQ_9,4M_HOLIDAY_UNIFIED_UQ_HOLIDAY,4M_CONTENT_CATEGORY_BOTTOM_UQ_6,4M_PAGE_HIERARCHY_CUENTAS,4M_PAGE_HIERARCHY_RRHH,4M_CONTENT_CATEGORY_BOTTOM_UQ_60,4M_PAGE_OWN_LOANS,4M_ON_SITE_SEARCH_TERM_UQ_228,4M_HOLIDAY_LOANS_UQ_NOT_HOLIDAY,4M_CONTENT_CATEGORY_6,4M_CONTENT_CATEGORY_3,4M_PAGE_HIERARCHY_ERROR,4M_ON_SITE_SEARCH_TERM_UQ_215,4M_ON_SITE_SEARCH_TERM_UQ_2,4M_ON_SITE_SEARCH_TERM_UQ_293,4M_CONTENT_CATEGORY_34,4M_ON_SITE_SEARCH_TERM_UQ_34,4M_PAGE_HIERARCHY_CONSUMOS-TARJETA-DEBITO,4M_CONTENT_CATEGORY_BOTTOM_13,4M_ON_SITE_SEARCH_TERM_44,4M_PAGE_HIERARCHY_UQ_MODIFICAR-CUENTA-PRIMARIA,...,6M_PAGE_HIERARCHY_ ESTADO DE SOLICITUDES,6M_ON_SITE_SEARCH_TERM_55,6M_ON_SITE_SEARCH_TERM_UQ_141,6M_ON_SITE_SEARCH_TERM_205,6M_CONTENT_CATEGORY_BOTTOM_25,6M_ON_SITE_SEARCH_TERM_UQ_6,6M_ON_SITE_SEARCH_TERM_UQ_100,6M_ON_SITE_SEARCH_TERM_UQ_64,6M_ON_SITE_SEARCH_TERM_UQ_111,6M_ON_SITE_SEARCH_TERM_42,6M_CONTENT_CATEGORY_BOTTOM_14,6M_ON_SITE_SEARCH_TERM_37,6M_CONTENT_CATEGORY_BOTTOM_UQ_14,6M_ON_SITE_SEARCH_TERM_UQ_261,6M_CONTENT_CATEGORY_11,6M_PAGE_HIERARCHY_UQ_CUENTAS,6M_CONTENT_CATEGORY_51,6M_PAGE_HIERARCHY_UQ_ EMINENT,6M_ON_SITE_SEARCH_TERM_UQ_177,6M_PAGE_HIERARCHY_UQ_BENEFICIOSYPROMOCIONES,6M_PAGE_HIERARCHY_UQ_INICIOGDFFGJ,6M_ON_SITE_SEARCH_TERM_UQ_76,6M_PAGE_HIERARCHY_CONSULTAS,6M_CONTENT_CATEGORY_BOTTOM_3,6M_ON_SITE_SEARCH_TERM_UQ_34,6M_ON_SITE_SEARCH_TERM_66,6M_ON_SITE_SEARCH_TERM_93,6M_CONTENT_CATEGORY_28,6M_ON_SITE_SEARCH_TERM_36,6M_ON_SITE_SEARCH_TERM_UQ_87,6M_ON_SITE_SEARCH_TERM_269,6M_ON_SITE_SEARCH_TERM_120,6M_ON_SITE_SEARCH_TERM_179,6M_CONTENT_CATEGORY_TOP_UQ_5,6M_ON_SITE_SEARCH_TERM_161,6M_ON_SITE_SEARCH_TERM_UQ_109,6M_CONTENT_CATEGORY_59,6M_PAGE_HIERARCHY_MENULINK,6M_ON_SITE_SEARCH_TERM_UQ_102,6M_CONTENT_CATEGORY_17
0,0,1,0,282.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,1.0,141.0,0.0,7746.0,0.0,0.0,282.0,1.0,0.0,9.0,1.0,845.0,NaN,NaN,0.0,0.0,26.0,141.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN,4.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0
1,1,1,0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,10000.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,NaN,NaN,0.0,0.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,2.0,0.0,1.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,1.0,0.0,NaN,NaN,NaN,NaN,0.0
2,2,1,0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,2362.0,0.0,3937.0,0.0,0.0,0.0,1.0,0.0,2.0,0.0,1575.0,NaN,NaN,0.0,0.0,11.0,0.0,0.0,2362.0,0.0,0.0,0.0,NaN,0.0,0.0,2362.0,0.0,0.0,...,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,1575.0,2.0,NaN,0.0,0.0,3.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,1.0,0.0,NaN,NaN,NaN,NaN,0.0
3,3,1,0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,1.0,21368.0,0.0,0.0,0.0,0.0,1.0,6.0,0.0,0.0,NaN,NaN,0.0,0.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,...,855.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0
4,4,1,0,7717.0,0.0,0.0,0.0,NaN,0.0,1.0,965.0,3.0,5788.0,0.0,965.0,0.0,0.0,965.0,4.0,0.0,7.0,4.0,0.0,NaN,NaN,0.0,0.0,17.0,1929.0,0.0,2894.0,0.0,0.0,0.0,NaN,0.0,0.0,322.0,0.0,0.0,...,1929.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,1.0,0.0,NaN,NaN,NaN,NaN,0.0


In [0]:
# 5 months time's window
historical_features_by_time_window(5, cc.PATH_DT_5MONTHS, 100, False)

[START] - Historical Features for 5 Months 
     Feature Build     [ 10% 20% 30% 40% 50% 60% 70% 80% 90% 100%  ] 
     Feature Selection [ 10% 20% 30% 40% 50% 60% 70% 80% 90% 100%  ] 
[ENDED] - Historical Features for 5 Months



## Trend Features - 4 vs 6 months
Now we will build TREND features which are a especial type of historical features where we consider the delta between two (o more) differents time's windows. In this case we will compare the 4 last months vs the 6 last months. What's more, we will build TREND features just for the features which both time's windows have. So, the features to create are:

`TREND46_XXCOL_XXVAL`

`TREND46_XXCOL_UQ_XXVAL`

In [0]:
# User-defined function to scale a feature base in the local max
def scale_feature(dt, cols):
    for c in cols:
        min_max_scaler = preprocessing.MaxAbsScaler()
        dt[c] = min_max_scaler.fit_transform(dt[[c]])
        dt[c] = (dt[c] * cc.K_MAX_BINS)
        dt[c].fillna((-1 * (1 + cc.K_MAX_BINS)),inplace=True)
        dt[c] = round(dt[c]).astype(int)

# User-defined function to calculte custom trend features for a t3 & t6 features based
def trend(t3, t6):
    return  int(round((t3/t6) * 10)) if (t6 > 0) else int(10) if (t3 >0) else None

# At first, we need to match the same columns in boths datasets
fes = set([ selected_features[4][i].lstrip("4M_") for i in range(len(selected_features[4]))]).intersection(
      set([ selected_features[6][i].lstrip("6M_") for i in range(len(selected_features[6]))]))

# Discard the index columns
x = [fes.discard(col) for col in ["USER_ID","MONTH","ONTH","TARGET"]]

# For each column which match, we will build the trend 3vs6 months feature
q_columns_trend = len(fes)
q_col_trend = 0
q_max3 = 0
dt_tw_unified2 = dt_tw_unified[["USER_ID","MONTH"] + ["4M_"+f for f in fes] + ["6M_"+f for f in fes]]
print("[START] - Trend Features: "+str(q_columns_trend)+" [", end = '')
for c in fes:
    q_col_trend = (q_col_trend+1)
    if int((q_col_trend/q_columns_trend)*10) > q_max3:
        q_max3 = int((q_col_trend/q_columns_trend)*10)
        print(str(int(q_max3*10))+"% ", end = '')
    
    # At first we need to normalize the 3 & 6 months features to make it comparable.
    scale_feature(dt_tw_unified, ["4M_"+c,"6M_"+c])
    
    # Then we build the trend features
    dt_tw_unified2["TREND46_"+c] = list(map(lambda t3, t6: trend(t3, t6), dt_tw_unified["4M_"+c], dt_tw_unified["6M_"+c]))
    
    # Drop de original columns
    dt_tw_unified2.drop(["4M_"+c,"6M_"+c], axis=1, inplace=True)

print("] [ENDED]")

[START] - Trend Features: 211 [10% 20% 30% 40% 50% 60% 70% 80% 90% 100% ] [ENDED]


In [0]:
dt_tw_unified2.to_csv(BASE_PATH + cc.PATH_DT_4M_6M)
dt_tw_unified2.head()

,USER_ID,MONTH,TREND46_HOLIDAY_LOANS_HOLIDAY_LOANS,TREND46_PAGE_HIERARCHY_UQ_PERSONAS,TREND46_ON_SITE_SEARCH_TERM_UQ_212,TREND46_PAGE_OWN_UQ_LOANS,TREND46_CONTENT_CATEGORY_TOP_UQ_2,TREND46_PAGE_HIERARCHY_UQ_INICIO,TREND46_ON_SITE_SEARCH_TERM_120,TREND46_CONTENT_CATEGORY_BOTTOM_2,TREND46_PAGE_HIERARCHY_UQ_ RESUMEN DE PRODUCTOS,TREND46_ON_SITE_SEARCH_TERM_UQ_34,TREND46_PAGE_HIERARCHY_INFORMACION-CONTACTO,TREND46_SITE_ID_UQ_2,TREND46_ON_SITE_SEARCH_TERM_UQ_219,TREND46_PAGE_OWN_UQ_REQUEST_LOAN,TREND46_ON_SITE_SEARCH_TERM_UQ_39,TREND46_SITE_ID_UQ_1,TREND46_WEEKEND_UQ_WEEK,TREND46_CONTENT_CATEGORY_TOP_5,TREND46_PAGE_OWN_UQ_LOG,TREND46_ON_SITE_SEARCH_TERM_1,TREND46_PAGE_HIERARCHY_RESUMEN,TREND46_WEEKEND_WEEKEND,TREND46_CONTENT_CATEGORY_BOTTOM_18,TREND46_PAGE_HIERARCHY_INICIO%7D,TREND46_ON_SITE_SEARCH_TERM_76,TREND46_ON_SITE_SEARCH_TERM_UQ_220,TREND46_CONTENT_CATEGORY_BOTTOM_59,TREND46_ON_SITE_SEARCH_TERM_77,TREND46_CONTENT_CATEGORY_BOTTOM_UQ_60,TREND46_CONTENT_CATEGORY_UQ_6,TREND46_PAGE_OWN_REQUEST_LOAN,TREND46_CONTENT_CATEGORY_BOTTOM_UQ_1,TREND46_ON_SITE_SEARCH_TERM_UQ_40,TREND46_PAGE_HIERARCHY_LANDING,TREND46_WEEKEND_WEEK,TREND46_PAGE_HIERARCHY_UQ_ SIDEBAR DERECHO,TREND46_PAGE_HIERARCHY_UQ_AYUDA,TREND46_SITE_ID_3,...,TREND46_CONTENT_CATEGORY_3,TREND46_ON_SITE_SEARCH_TERM_179,TREND46_PAGE_OWN_UQ_TRANSFERS,TREND46_PAGE_HIERARCHY_CONSUMOS-TARJETA-DEBITO,TREND46_ON_SITE_SEARCH_TERM_UQ_23,TREND46_PAGE_HIERARCHY_MENSAJE,TREND46_CONTENT_CATEGORY_7,TREND46_ON_SITE_SEARCH_TERM_44,TREND46_CONTENT_CATEGORY_16,TREND46_ON_SITE_SEARCH_TERM_55,TREND46_PAGE_HIERARCHY_UQ_CONSUMOS,TREND46_CONTENT_CATEGORY_BOTTOM_UQ_2,TREND46_PAGE_HIERARCHY_ERROR,TREND46_PAGE_HIERARCHY_ RESUMEN DE PRODUCTOS,TREND46_PAGE_HIERARCHY_DOMICILIO,TREND46_PAGE_HIERARCHY_UQ_CUENTASPROYECTO,TREND46_PAGE_HIERARCHY_UQ_CUENTAS,TREND46_WEEKEND_UQ_WEEKEND,TREND46_PAGE_HIERARCHY_CUENTASPROYECTO,TREND46_ON_SITE_SEARCH_TERM_UQ_41,TREND46_HOLIDAY_OPEN_HOLIDAY,TREND46_PAGE_HIERARCHY_UQ_SEGUROS,TREND46_PAGE_HIERARCHY_UQ_WEB,TREND46_PAGE_OWN_LOANS,TREND46_ON_SITE_SEARCH_TERM_248,TREND46_CONTENT_CATEGORY_BOTTOM_9,TREND46_CONTENT_CATEGORY_UQ_59,TREND46_PAGE_HIERARCHY_UQ_INFORMACION-CONTACTO,TREND46_PAGE_HIERARCHY_ SIDEBAR DERECHO,TREND46_PAGE_HIERARCHY_INSTITUCIONAL,TREND46_CONTENT_CATEGORY_BOTTOM_13,TREND46_SITE_ID_1,TREND46_HOLIDAY_LOANS_UQ_HOLIDAY_LOANS,TREND46_PAGE_HIERARCHY_PAGOS,TREND46_CONTENT_CATEGORY_UQ_4,TREND46_CONTENT_CATEGORY_BOTTOM_16,TREND46_PAGE_HIERARCHY_ PASO 4,TREND46_CONTENT_CATEGORY_BOTTOM_6,TREND46_PAGE_HIERARCHY_UQ_VISAHOME,TREND46_ON_SITE_SEARCH_TERM_115
0,0,1,NaN,NaN,NaN,NaN,15.0,15.0,NaN,10.0,15.0,NaN,NaN,15.0,NaN,NaN,NaN,15.0,15.0,NaN,15.0,10.0,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.0,NaN,15.0,NaN,NaN,10.0,14.0,NaN,10.0,...,NaN,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.0,15.0,NaN,10.0,NaN,NaN,20.0,14.0,NaN,NaN,10.0,NaN,15.0,NaN,NaN,NaN,NaN,NaN,10.0,NaN,NaN,10.0,NaN,10.0,15.0,NaN,NaN,NaN,NaN,NaN
1,1,1,NaN,NaN,NaN,NaN,15.0,NaN,NaN,10.0,15.0,NaN,NaN,15.0,NaN,NaN,NaN,15.0,15.0,10.0,15.0,10.0,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.0,NaN,NaN,10.0,14.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.0,NaN,10.0,NaN,NaN,NaN,14.0,NaN,NaN,10.0,NaN,15.0,NaN,NaN,NaN,NaN,NaN,10.0,NaN,NaN,10.0,NaN,NaN,15.0,NaN,NaN,NaN,NaN,NaN
2,2,1,NaN,NaN,NaN,NaN,15.0,13.0,NaN,10.0,16.0,NaN,NaN,15.0,NaN,NaN,NaN,14.0,15.0,10.0,15.0,10.0,9.0,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.0,NaN,NaN,10.0,NaN,NaN,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.0,15.0,10.0,10.0,NaN,NaN,15.0,15.0,NaN,NaN,10.0,NaN,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,10.0,NaN,NaN,15.0,NaN,NaN,NaN,NaN,NaN
3,3,1,NaN,13.0,NaN,NaN,15.0,NaN,NaN,10.0,15.0,NaN,NaN,15.0,NaN,NaN,NaN,15.0,15.0,NaN,15.0,10.0,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.0,NaN,NaN,10.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.0,NaN,10.0,NaN,NaN,NaN,14.0,NaN,NaN,10.0,NaN,15.0,NaN,NaN,8.0,NaN,NaN,NaN,NaN,NaN,10.0,NaN,NaN,15.0,NaN,NaN,NaN,NaN,NaN
4,4,1,NaN,NaN,NaN,NaN,15.0,13.0,NaN,10.0,15.0,NaN,NaN,14.0,Na

In [0]:
# Delete the datasets used
del dt_tw_unified
del dt_tw_unified2

# The End 